In [1]:
import cv2
import numpy as np

buttons = {
    'Clear': ((10, 10), (100, 50), (200, 200, 200)),
    'Red': ((110, 10), (180, 50), (0, 0, 255)),
    'Green': ((190, 10), (260, 50), (0, 255, 0)),
    'Blue': ((270, 10), (340, 50), (255, 0, 0)),
    'Black': ((350, 10), (420, 50), (0, 0, 0)),
    "Ereser":((430,10),(500,50),(255,217,22)),
    "Rectangle":((510,10),(610,50),(0,135,255)),
    "Bresh":((620,10),(690,50),(200,10,100)),
    "Line":((700,10),(770,50),(100,200,10)),
    "Circle":((780,10),(850,50),(10,100,200)),
    "Ellipse":((860,10),(930,50),(200,150,50)),
    "Save":((940,10),(1010,50),(255,0,255))
}
            
def draw_buttons(img):
    for name, ((x1, y1), (x2, y2), bgr) in buttons.items():
        cv2.rectangle(img, (x1, y1), (x2, y2), bgr, -1)
        cv2.putText(img, name, (x1 + 5, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255) if bgr != (200,200,200) else (0,0,0), 2)
    
color = (0,0,0)
mode = ""
def check_button_click(x, y, b,g,r):
    global color, canvas,mode
    for name, ((x1, y1), (x2, y2), bgr) in buttons.items():
        if x1 <= x <= x2 and y1 <= y <= y2:
            if name == 'Clear':
                canvas[:, :] = 255
                return canvas
            elif name == "Red":
                update_trackbars_from_color(0,0,255)
            elif name == "Green":
                update_trackbars_from_color(0,255,0)
            elif name == "Blue":
                update_trackbars_from_color(255,0,0)
            elif name == "Black":
                update_trackbars_from_color(0,0,0)
            elif name == "Ereser":
                mode = "Bresh"
                update_trackbars_from_color(255,255,255)
            
            elif name == "Rectangle":
                mode = "Rectangle"
            
            elif name == "Bresh":
                mode = "Bresh"
                
            elif name == "Line":
                mode = "Line"
                
            elif name == "Circle":
                mode = "Circle"
                
            elif name == "Ellipse":
                mode = "Ellipse"
            elif name == "Save":
                img = canvas[55:,:]
                cv2.imwrite("Painting.jpg",img)
            else:
                color = bgr
                return color
            
def update_trackbars_from_color(b,g,r):
    cv2.setTrackbarPos("Blue","paint",b)
    cv2.setTrackbarPos("Green","paint",g)
    cv2.setTrackbarPos("Red","paint",r)
    
temp_canvas = 0
ix, iy = -1, -1 
d = 4
def f1(event, x, y, flags, param):
    global ix, iy, d, mode, temp_canvas
    if y < 50:
        if event == cv2.EVENT_LBUTTONDOWN:
            check_button_click(x, y,b,g,r)
    else:
        if mode == "Bresh":
            if event ==cv2.EVENT_LBUTTONDOWN:
                d = 2
                ix, iy = x, y
            elif event == cv2.EVENT_MOUSEMOVE:
                 if d == 2:
                     cv2.line(canvas, (ix, iy), (x, y), [b,g,r], br)
                     ix, iy = x, y
            elif event == cv2.EVENT_LBUTTONUP:
                d = 4
                
        elif mode == "Rectangle":
            if event == cv2.EVENT_LBUTTONDOWN:
                d = 2
                ix, iy = x, y
                temp_canvas = canvas.copy()
            elif event == cv2.EVENT_MOUSEMOVE:
                if d == 2:
                    canvas[:] = temp_canvas.copy()  
                    cv2.rectangle(canvas, (ix, iy), (x, y), [b, g, r], br)
            elif event == cv2.EVENT_LBUTTONUP:
                d = 4
                
        elif mode == "Line":
            if event == cv2.EVENT_LBUTTONDOWN:
                d = 2
                ix, iy = x, y
                temp_canvas = canvas.copy() 
            elif event == cv2.EVENT_MOUSEMOVE:
                if d == 2:
                    canvas[:] = temp_canvas.copy() 
                    cv2.line(canvas, (ix, iy), (x, y), [b, g, r], br)  
            elif event == cv2.EVENT_LBUTTONUP:
                d = 4

        elif mode == "Circle":
            if event == cv2.EVENT_LBUTTONDOWN:
                d = 2
                ix, iy = x, y
                temp_canvas = canvas.copy() 
            elif event == cv2.EVENT_MOUSEMOVE:
                if d == 2:
                    canvas[:] = temp_canvas.copy() 
                    radius = int(((x - ix)**2 + (y - iy)**2)**0.5)
                    cv2.circle(canvas, (x, y), radius, [b, g, r], br) 
            elif event == cv2.EVENT_LBUTTONUP:
                d = 4
        elif mode == "Ellipse":
            if event == cv2.EVENT_LBUTTONDOWN:
                d = 2
                ix, iy = x, y
                temp_canvas = canvas.copy() 
            elif event == cv2.EVENT_MOUSEMOVE:
                if d == 2:
                    canvas[:] = temp_canvas.copy()
                    axis_x = abs(x - ix)
                    axis_y = abs(y - iy)
                    center = ((ix + x) // 2, (iy + y) // 2)
                    cv2.ellipse(canvas, center, (axis_x // 2, axis_y // 2), 0, 0, 360, (b, g, r), br)
            elif event == cv2.EVENT_LBUTTONUP:
                d = 4

cv2.namedWindow("paint")
cv2.setMouseCallback("paint",f1)
cv2.createTrackbar("Blue","paint",0,255,lambda x:None)
cv2.createTrackbar("Green","paint",0,255,lambda x:None)
cv2.createTrackbar("Red","paint",0,255,lambda x:None)
cv2.createTrackbar("bresh Thickness","paint",2,20,lambda x:None)
canvas = np.full((1000,1300,3),255).astype(np.uint8)

while True:
    b = cv2.getTrackbarPos("Blue","paint")
    g = cv2.getTrackbarPos("Green","paint")
    r = cv2.getTrackbarPos("Red","paint")
    br = cv2.getTrackbarPos("bresh Thickness","paint")+2
    draw_buttons(canvas)
    cv2.rectangle(canvas,(1190,10),(1250,50),(b,g,r),-1)
    cv2.putText(canvas, "Color", (1195,40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    cv2.imshow("paint",canvas)
    press = cv2.waitKey(1)
    
    if press == 27:
        break

cv2.destroyAllWindows()